In [ ]:
from transformers import pipeline
import torch

In [ ]:
model_id = "openai/gpt-oss-20b"

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype="auto",
    device_map="auto",    
)

messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)

In [ ]:
print(outputs[0]["generated_text"][-1])

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client=OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN")
)

response = client.chat.completions.create(
    model= "openai/gpt-oss-120b:cerebras",
    messages=[{"role": "user", "content":"Tell me a fun fact about the Eiffel Tower."}]
)

print(response.choices[0].message.content)

Here's a fun tidbit: **the Eiffel Tower has a secret “bicycle‑rental” office hidden in its very first floor!**  

When Gustave Eiffel completed the tower in 1889, he wanted to make the site a lively attraction, not just a static monument. So he set up a little “Bicycle Rental” (in French, *Location de Vélos*) right on the tower’s ground level. Early visitors could rent a bike and pedal around the Champ de Mars, taking in the views of the tower from the ground up—something like a primitive “tourist package” that combined transport and sightseeing.

The office was short‑lived (it closed after a few years), but the tiny door is still there, tucked into a corner of the first‑floor lobby. If you look closely during a visit, you can spot the faded sign above the door, a quiet reminder of the tower’s early‑20th‑century effort to turn a massive iron lattice into a full‑on tourist experience. 🚴‍♂️✨


In [5]:
import os
from openai import OpenAI

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client.chat.completions.create(
    model="openai/gpt-oss-120b:cerebras",
    messages=[{"role": "user", "content": "What is the weather in Paris in Celsius?"}],
    tools=tools,
    tool_choice="auto",
)

# The response will contain the tool_calls object if the model decides to use the tool
print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='afde8712c', function=Function(arguments='{\n  "location": "Paris, FR",\n  "unit": "celsius"\n}', name='get_current_weather'), type='function')], reasoning='User asks for weather in Paris in Celsius. Need to call get_current_weather with location "Paris, FR" perhaps. The function expects "city and state" like "San Francisco, CA". But we can give "Paris, FR". Provide unit "celsius". Use function.')
